In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [23]:
# Load API Credentials
with open('/Users/Testing/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [24]:
# set our API call parameters 
LOCATION = 'Boston'
TERM = 'burger'

In [25]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_boston_burger.json"
JSON_FILE

'Data/results_in_progress_boston_burger.json'

This function accepts the JSON_FILE filename as first argument and a second argument called delete_if_exists and set to it to False by default.

This way, it will not automatically delete previous search results. We will have to explicitly say delete_if_exists = True to do so.

In [26]:
# This functionwill not automatically delete previous search results. We will have to explicitly say delete_if_exists = True to do so.
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            create_json_file(JSON_FILE,delete_if_exists=False)  #Recursive call to function after old file deleted
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE) 
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:             
            # create the folder
            os.makedirs(folder,exist_ok=True)  # 'Data/'
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)          


In [27]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_boston_burger.json not found. Saving empty list to new file.
- 0 previous results found.


110

In [28]:
# first call
for i in tqdm_notebook(range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results) 
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/110 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [29]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,Dvwj6KWrDZHVQwlSjYS4Iw,boston-burger-boston-2,Boston Burger,https://s3-media4.fl.yelpcdn.com/bphoto/lyJDpK...,False,https://www.yelp.com/biz/boston-burger-boston-...,1197,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,"{'latitude': 42.34691395544883, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '1100 Boylston St', 'address2': '...",+18572334560,(857) 233-4560,1423.494923
1,xayS1Ht3N8g_YBe1wdEDCg,the-capital-burger-boston,The Capital Burger,https://s3-media4.fl.yelpcdn.com/bphoto/2b0Xsc...,False,https://www.yelp.com/biz/the-capital-burger-bo...,186,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"{'latitude': 42.35087419, 'longitude': -71.078...",[],$$,"{'address1': '159 Newbury St', 'address2': '',...",+16172621333,(617) 262-1333,688.898491
2,Hc9l2eLAGTB4lZJI3vRoPw,tasty-burger-boston,Tasty Burger,https://s3-media2.fl.yelpcdn.com/bphoto/piJd-i...,False,https://www.yelp.com/biz/tasty-burger-boston?a...,1255,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,"{'latitude': 42.3445778, 'longitude': -71.0982...",[delivery],$,"{'address1': '1301 Boylston St', 'address2': '...",+16174254444,(617) 425-4444,2258.810410
3,ncBpwiVsW61DLoJAhTwgfg,lees-burger-place-newton-centre-2,Lee's Burger Place,https://s3-media4.fl.yelpcdn.com/bphoto/tRw_Ph...,False,https://www.yelp.com/biz/lees-burger-place-new...,359,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"{'latitude': 42.3307251, 'longitude': -71.1922...","[delivery, pickup]",$,"{'address1': '216 Sumner St', 'address2': '', ...",+16177952022,(617) 795-2022,10122.398262
4,XdTb4p34mtbxG9Dyjk25hQ,crazy-good-kitchen-newbury-boston,Crazy Good Kitchen - Newbury,https://s3-media1.fl.yelpcdn.com/bphoto/KaBF0E...,False,https://www.yelp.com/biz/crazy-good-kitchen-ne...,158,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 42.34914, 'longitude': -71.08327}","[delivery, pickup]",$$,"{'address1': '268 Newbury St', 'address2': Non...",+16179364359,(617) 936-4359,999.355858


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,F9CV9OuIhW-pm-PKKFXVBA,finch-boston,Finch,https://s3-media2.fl.yelpcdn.com/bphoto/L9Bsdu...,False,https://www.yelp.com/biz/finch-boston?adjust_c...,32,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",3.5,"{'latitude': 42.36389, 'longitude': -71.06241}",[delivery],$$,"{'address1': '107 Merrimac St', 'address2': 'T...",+16176240202,(617) 624-0202,1922.653046
996,v95lM5eqV1feLxQakFN5bA,wagamama-prudential-boston,wagamama - Prudential,https://s3-media2.fl.yelpcdn.com/bphoto/TBzNjk...,False,https://www.yelp.com/biz/wagamama-prudential-b...,742,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",3.5,"{'latitude': 42.347632, 'longitude': -71.081682}","[delivery, pickup]",$$,"{'address1': '800 Boylston St', 'address2': ''...",+16177782344,(617) 778-2344,858.178211
997,4m8ZezBnKeDYf9WIqJhAHw,clover-food-lab-burlington-2,Clover Food Lab,https://s3-media3.fl.yelpcdn.com/bphoto/FqqFjG...,False,https://www.yelp.com/biz/clover-food-lab-burli...,360,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.5,"{'latitude': 42.48626363501841, 'longitude': -...",[],$$,"{'address1': '100 Burlington Mall Rd', 'addres...",,,19270.191356
998,Ta0fmDvhG7SjcAjADFTntw,savvor-restaurant-and-lounge-boston,Savvor Restaurant and Lounge,https://s3-media3.fl.yelpcdn.com/bphoto/HDPxuv...,False,https://www.yelp.com/biz/savvor-restaurant-and...,97,"[{'alias': 'lounges', 'title': 'Lounges'}, {'a...",3.0,"{'latitude': 42.350483, 'longitude': -71.058691}","[delivery, pickup]",$$,"{'address1': '180 Lincoln St', 'address2': Non...",+18572502165,(857) 250-2165,1072.529540
999,bxy3khT-2R66tcdKjFa2pw,posto-somerville,Posto,https://s3-media3.fl.yelpcdn.com/bphoto/NMfM5G...,False,https://www.yelp.com/biz/posto-somerville?adju...,1016,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 42.39317, 'longitude': -71.12024}","[delivery, pickup]",$$,"{'address1': '187 Elm St', 'address2': '', 'ad...",+16176250600,(617) 625-0600,6444.743895


In [31]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [32]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_boston_burger.csv.gz', compression='gzip',index=False)